# Fundamentals of Machine Learning - Exercise 10
Goal of the excercise is to learn how to use Scikit-learn library for a regression tasks employing various linear regression models and moreover evaluate the performance of the proposed models.

![meme01](https://github.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/blob/master/images/fml_10_meme_01.jpeg?raw=true)

## 📌 Useful URLs

### Models
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

### Preprocessing
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

In [ ]:
"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, rmse, mape = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), np.sqrt(mean_squared_error(y_true=y_true, y_pred=y_pred)), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape}], index=[0])

## Petrol Consumption Dataset
https://www.kaggle.com/datasets/harinir/petrol-consumption

### 🎯 Our goal is to build a regression model for prediction of petrol consumption in the 48 USA states.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/master/datasets/petrol_consumption.csv')
df.head()

## Is each column numerical?

In [ ]:
df.dtypes

## Do we have any missing data?

In [ ]:
df.isna().sum()

# 📊 Let's start with a simple EDA

* 🔎 Can you see any relationships among the features from the pairplot?
    * What should we look for?
* 🔎 Do you think that the features are normally distributed?

In [ ]:
fig = plt.figure(figsize=(12, 12))
sns.pairplot(df)
fig.show()

## Always look for a simple trend-like patters first 🙂
> ## **Trend is your friend** 😀

![meme02](https://github.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/blob/master/images/fml_10_meme_02.png?raw=true)

## What about the a correlation coefficients?
* 🔎 What row/column is the most important from the heatmap?
    * Why?
* 🔎 Are correlations among **independent variables** good or bad?

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(df.corr(), square=True, cmap='RdYlGn', vmin=-1, vmax=1, annot=True)

## Can you see any outliers in the data?
* What about skewness or variance differences?

In [ ]:
fig, axes = plt.subplots(1, df.shape[1], figsize=(10, 5))

for i, col in enumerate(df.columns):
    ax = axes.flatten()[i]
    sns.boxplot(data=df, y=col, ax=ax)
        
fig.tight_layout()

# 🚀 Let's build our first simple regression models with just 2 variables and compare them
* We will split the data into train/test set
* Then we can build the models and evaluate them

### There are many metrics used for the perormance evaluation
* MAE, RMSE, MAPE, R2, etc.
    * Do you know what these abbr. mean?
* 🔎 **Do we want these metrics to go lower or higher?**
    * Is it the same direction as in classification tasks, e.g. F1-Score, or opposite way around? 
* 💡 You can take a look at these blog posts:
    * [this](https://towardsdatascience.com/regression-an-explanation-of-regression-metrics-and-what-can-go-wrong-a39a9793d914)
    * or [this](https://www.analyticsvidhya.com/blog/2021/05/know-the-best-evaluation-metrics-for-your-regression-model/) for more details

## Create `X` and `y` dataframes

In [ ]:
X, y = df.drop('Petrol_Consumption', axis=1), df['Petrol_Consumption']

In [ ]:
X.head()

In [ ]:
y.head()

## Split the data in ration 80:20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

# ⚡ The 1st model will be the simplest one
* We will choose only one feature for the model - *Population_Driver_licence(%)*
    * 🔎 Why did we chose this specific feature?

In [ ]:
s_column = 'Population_Driver_licence(%)'
alg = LinearRegression()
alg.fit(pd.DataFrame(X_train[s_column]), y_train)

In [ ]:
y_pred = alg.predict(pd.DataFrame(X_test[s_column]))

## 🔎 How would the regression line formula look like?
* 💡 What is a general equation of straight line in 2D? And for nD?

In [ ]:
alg.coef_[0]

In [ ]:
alg.intercept_

# 💡 Very simple visual check of prediction quality is `y_test vs. y_pred` scatter plot
* What is an ideal result?

In [ ]:
sns.scatterplot(x=y_test, y=y_pred)

# However it is always better to quantify the errors 😊
* 💡MAPE or SMAPE uses percentage values, thus these might be easier to understand to non-expert audience
* 💡MAE, RMSE are in the same units as the predicted variable
    * Always take a look at basic statistical properties (typical value range, variance or use box-plot ) to rationalize the amount of error according to the range or the variable
    * 📌 e.g., MAE = 10 can be low for variable in <1000, 5000> range but very high for variable in <0, 50> range

In [ ]:
sns.boxplot(y=y)

## So what do you think about the error?
* 💡 Is model completely off or is it roughly right?

In [ ]:
compute_metrics_raw(y_true=y_test, y_pred=y_pred)

# ⚡ 2nd model will use just one variable again, however now it will be an uncorrelated one
* 🎯 We want to compare the model with the 1st one

In [ ]:
s_column = 'Paved_Highways'
alg = LinearRegression()
alg.fit(pd.DataFrame(X_train[s_column]), y_train)

In [ ]:
y_pred = alg.predict(pd.DataFrame(X_test[s_column]))

## Let's take a look at the scatterplot of y_test vs. y_pred now

In [ ]:
sns.scatterplot(x=y_test, y=y_pred)

## 🔎 Which one of the two models is better and why?

In [ ]:
compute_metrics_raw(y_true=y_test, y_pred=y_pred)

# The obvious next step is using more than one feature in the model, so let's get to it! 👊
* The API is the same, we will just include every feature in the model instead of just one

In [ ]:
alg = LinearRegression()
alg.fit(X_train, y_train)

In [ ]:
y_pred = alg.predict(X_test)

## How would the regression line formula look like now?

In [ ]:
alg.coef_

In [ ]:
alg.intercept_

## 📊 For MLR we usually also want to take a look at coefficients values so we can "explain" the decisions by the model
* 🔎 Are all the features used?
* 🔎 Is there any feature much more important than other features?

In [ ]:
sns.barplot(x=X_train.columns, y=alg.coef_)
plt.xticks(rotation=30)

## 🔎 Is the model better than the 1st one with just one feature?
* How different are the results?

In [ ]:
compute_metrics_raw(y_true=y_test, y_pred=y_pred)

## 🔎 Is it wise to have a model with some coefficient of few magnitudes higher values than other coefficients?
* What can go wrong? 
*  What is a **colinearity?** 
    * Why it may become an issue for regression models?

![meme03](https://github.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/blob/master/images/fml_10_meme_03.jpg?raw=true)

# There are method for dealing with of these issues
* It is called regularization
    * We have two types of it - **L1 (Lasso)** and **L2 (Ridge)**
    * What is the difference between them?
* How is the regularization used?
    * What do we change in the model?
 
* Very nice comparison of both methods is at https://www.datacamp.com/tutorial/tutorial-lasso-ridge-regression

# Let's try L1 - Lasso first
* 💡 The most important parameter is the `alpha` value
* Higher alpha means that the regularization will be more strict

In [ ]:
alg = Lasso(alpha=3, random_state=13)
alg.fit(X_train, y_train)

In [ ]:
y_pred = alg.predict(X_test)

## 💡 Notice the values of coefficients

In [ ]:
alg.coef_

In [ ]:
alg.intercept_

In [ ]:
sns.barplot(x=X_train.columns, y=alg.coef_)
plt.xticks(rotation=30)

In [ ]:
compute_metrics_raw(y_true=y_test, y_pred=y_pred)

# We can use L2 - Ridge in the same way

In [ ]:
alg = Ridge(alpha=1, random_state=13)
alg.fit(X_train, y_train)

In [ ]:
y_pred = alg.predict(X_test)

In [ ]:
alg.coef_

In [ ]:
alg.intercept_

In [ ]:
sns.barplot(x=X_train.columns, y=alg.coef_)
plt.xticks(rotation=30)

In [ ]:
compute_metrics_raw(y_true=y_test, y_pred=y_pred)

### 🔎 Regardless the used model, what is the difference among the coefficients values with enabled and disabled regularization?

# 💡 There are usually differences among the variables ranges 
* This may bring some difficulties in the coefficient optimization process
* 💡 If the ranges are similar, the optimization process should be a lot easier
    * Why?
* Due to that, we usually use `MinMaxScaler` or `StandardScaler` before we try to fit a linear regression model
    * We are not limited to these two preprocessing methods

# 🚀 We will try to fit the Lasso model again, but this time with scaled features

### 🔎 Why do we fit the scaler only on the training part of the data?

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_std = std_scaler.transform(X_train)
X_test_std = std_scaler.transform(X_test)

### Try 0.1, 1 and 10 for alpha parameter
* What is different for each run?

In [ ]:
def lasso_experiments(X_train_std, y_train, X_test_std, y_pred, alphas=[0.1, 1, 10]):
    results = []
    for alpha in alphas:
        alg = Lasso(alpha=alpha, random_state=13)
        alg.fit(X_train_std, y_train)
        y_pred = alg.predict(X_test_std)
        results.append((alpha, y_pred, alg.coef_))
    return results

In [ ]:
alphas = [0.1, 1, 10]
results = lasso_experiments(X_train_std, y_train, X_test_std, y_pred, alphas)

## 🔎 How are the models and results different?

In [ ]:
df_results = pd.DataFrame(index = alphas, columns=['MAE', 'MSE', 'RMSE', 'MAPE'])
for i, x in enumerate(results):
    df_results.iloc[i, :] = compute_metrics_raw(y_true=y_test, y_pred=x[1])
df_results

In [ ]:
df_coeffs = pd.DataFrame(index = alphas, columns=X_train.columns)
for i, x in enumerate(results):
    df_coeffs.iloc[i, :] = x[2]
df_coeffs

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
for i, x in enumerate(results):
    ax = sns.barplot(x=X_train.columns, y=x[2], ax=axs[i])
    ax.tick_params(axis='x', rotation=45)

# ✅ Task (2p)
* We are obviously not limited to only a linear regression models for the regression tasks
    * Usually there is a regression alternative for most of the classification models in Sk-Learn
* Use [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) for the data and compare it to the Linear regression model (**1p**)
    * 💡 Plot the *feature_importances_* if you want to know which features are important for this model
    * More info [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
* Use [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) model
    * The model combines the L1 and L2 regularization
    * Study how the model works
    * The model has 2 important parameters `alpha` and `l1_ratio`
    * Try to tune them
* Compare the `RandomForestRegressor` and `ElasticNet` models - which of them was more precise? (**1p**)

* Document everything you do in a Markdown cells
    * ❌ Results interpretation figured in real-time during task check is not allowed! ❌